#Billedeklassificering med Linear classifiers

I denne notebook arbejdes der med implementering og træning af lineære klassifikationsmodeller. Når I er færdige så skulle i gerne have fået indsigt i hvordan en parametrisk model er opbygget. Herunder hvordan loss/cost funktionen udregnes og hvordan gradienten af modellens parameter i forhold til dette bruges til at optimere modellen vha. af gradient decent.

Som i assignment1-1 er det tilladt at bruge ChatGPT, CO-Pilot etc. til at løse opgaven.

#Datasættet
Ligesom for kNN så anvendes der i denne notebook data fra "BloodMNIST" datasættet som kommer fra [MedMNIST databasen](https://medmnist.com/)

#Beskrivelse fra oprindelig kilde:
The BloodMNIST is based on a dataset of individual normal cells, captured from individuals without infection, hematologic or oncologic disease and free of any pharmacologic treatment at the moment of blood collection. It contains a total of 17,092 images and is organized into 8 classes. We split the source dataset with a ratio of 7:1:2 into training, validation and test set.

Billederne er tilgængelige i forskellige opløsninger 28x28, 64x64, 128x128,og 224x224 med 3 kanaler (RGB). I denne notebook arbejder vi med billeder i opløsning 28x28x3. De 8 forskellige klasser er:

['basophil', 'eosinophil', 'erythroblast', 'immature granulocytes', 'lymphocyte', 'monocyte', 'neutrophil', 'platelet']


Angivet kategorisk med værdierne:

[0, 1, 2, 3, 4, 5, 6, 7]

Nedenfor downloades og importeres datasættet og opdeles i separate trænings- og valideringssæt (tuning) og nogle eksempler illustreres.

In [ ]:
%pip install medmnist
import medmnist

from medmnist import BloodMNIST
trainDataset = BloodMNIST(split="train", download=True,size=28)
valDataset = BloodMNIST(split="val", download=True,size=28)
testDataset = BloodMNIST(split="test", download=True,size=28)

In [ ]:
trainImages,trainLabels,trainInfo = trainDataset.__dict__['imgs'],trainDataset.__dict__['labels'],trainDataset.__dict__['info']['label']


print('Træningsdata:')
print(f'Billeder: {trainImages.shape}, Labels: {trainLabels.shape}')


valImages,valLabels = valDataset.__dict__['imgs'],valDataset.__dict__['labels']
print('Valideringsdata:')
print(f'Billeder: {valImages.shape}, Labels: {valLabels.shape}')

testImages,testLabels = testDataset.__dict__['imgs'],testDataset.__dict__['labels']
print('Testdata:')
print(f'Billeder: {testImages.shape}, Labels: {testLabels.shape}')

import matplotlib.pyplot as plt
import random

print('\n')
print('plot nogle eksempler:')

random.seed(42)
fig, axes = plt.subplots(5, len(trainInfo), figsize=(15, 5))

for class_,name in trainInfo.items():
    print(f'klasse: {class_}, klassenavn: {name}. Antal træning samples: {len(trainLabels[trainLabels==int(class_)])}')
    # Get indices of all images belonging to class i
    class_indices = [idx for idx, label in enumerate(trainLabels) if int(class_) == label]
    # Randomly select 5 indices
    selected_indices = random.sample(class_indices, 5)
    for j, idx in enumerate(selected_indices):
        image, label = trainImages[idx],trainLabels[idx]
        axes[j, int(class_)].imshow(image, cmap='gray')
        axes[j, int(class_)].axis('off')
        if j == 0:
            axes[j, int(class_)].set_title(f'{name[:5]}: {class_}')

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np

#set random seed for reproducibility
np.random.seed(0)

#Shuffle the data
randomize = np.arange(trainImages.shape[0])
np.random.shuffle(randomize)

X_train = trainImages[randomize]
y_train = trainLabels[randomize].flatten()

randomizeVal = np.arange(valImages.shape[0])
X_val = valImages[randomizeVal]
y_val = valLabels[randomizeVal].flatten()


# Subsample the data for more efficient code execution in this exercise
num_training = 5000
mask = list(range(num_training))

X_train = X_train[mask]
y_train = y_train[mask]

print('first 10 examples in train: ',y_train[:10])

num_val = 500
mask = list(range(num_val))
X_val = X_val[mask]
y_val = y_val[mask]

print('first 10 examples in val: ',y_val[:10])

# Reshape the image data into rows for effecient distance calculation
#(vi tager billedet med dimensioner 28x28x3 og strækker det ud til en vektor med længden 28*28*3 = 2352)
X_train = np.reshape(X_train, (X_train.shape[0], -1))
X_val = np.reshape(X_val, (X_val.shape[0], -1))
print(f'New train shape: {X_train.shape}')
print(f'New val shape: {X_val.shape}')


In [ ]:
# Preprocessing: subtract the mean image
# first: compute the image mean based on the training data
mean_image = np.mean(X_train, axis=0)
print(mean_image[:10]) # print a few of the elements
plt.figure(figsize=(4,4))
plt.imshow(mean_image.reshape((28,28,3)).astype('uint8')) # visualize the mean image
plt.show()

# second: subtract the mean image from train and test data
X_train = X_train.astype(np.float32)-mean_image
X_val = X_val.astype(np.float32)-mean_image

print(X_train.min(),X_train.max())

# third: append the bias dimension of ones (i.e. bias trick) so that our SVM
# only has to worry about optimizing a single weight matrix W.
X_train = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
X_val = np.hstack([X_val, np.ones((X_val.shape[0], 1))])

print(X_train.shape, X_val.shape)

##Standadizing data
You could consider standardizing you data, e.g divide by 255 to get it on a scale from [-1,1]. This may help to avoid exploding gradients since the current values of the input data are quite large.

#Implementation of SVM/Hinge-loss
Below, implement a function to for calculating the hinge loss based on an image/set of images and associated class/label

In [ ]:
#################################################
#generate a random SVM weight matrix of small
#numbers corresponding to image size (+ bias dimension)
#depth of num classes
##################################################
image_size = 28*28*3+1
num_classes = 8

W = np.random.randn(image_size, num_classes) * 0.0001

###################################################
#The function should accept The W weight-/parameter
#matrix along with an image/batch of images as input
#and return the loss for the
#image/batch based on the claculated scores
#The l2 regularization loss should also be included
#in the function's claculation.
#In order to check the implementation, try to claculate
#the loss but with, and without regularization.
#For the hinge/svm loss, the expected loss from random
#initialized wieghts ~= k-1 where k is number of classes
####################################################

def svm_loss(W,X,y,reg=0):
  pass



#Test the loss using the validation set

E.g.:

    loss = svm_loss(W,X,y,reg):

In [ ]:
#Test the loss with and without regularization using the validation set
#With regularization the loss should increase.
#test with large value. e.g. 1e3 to make sure


#Compute the gradients of the weights with respect to the loss
In order to optimize the classifier we need to compute gradients for the weight matrix so the loss can be minimized. It is would be a good idea to include the gradient calculation in the loss function defined above such that scores, loss and gradient can be calculated at the same time using the function.

E.g.:

    loss,grad = svm_loss(W,X,y,reg)


The gradients are a matrix with the same shape a W. The regularization should also affect the gradient so make sure this is included in the computation




In [ ]:
###################################################
#Once the implementation is complete try to print the
#shape of the gradient matrix as well as the first row
#It should return something similar to this:
# [-3.0962016  4.9891232 -4.0875872  0.8926112  1.0465696 -1.5291232
#  3.9497376 -2.1651296]
#####################################################


#Implementation of Softmax-loss
Below, repeat the steps from above but for the softmax loss/gradient instead

In [ ]:
def softmax_loss(W,X,y,reg=0):
  pass

Test the loss and gradient calculations

E.g.:

    loss,grad = softmax_loss(W,X,y,reg)

The loss with an randomly initialized weight matrix should be ~ -ln(1/k) where k=number of classes

In [ ]:
###################################################
#Once the implementation is complete try to print the
#loss, shape of the gradient matrix as well as the first row
#The gradient claculation should return something similar to this:
# [-0.36533844  0.69120216 -0.47791155  0.10853622  0.05629067 -0.23380291
#  0.50159745 -0.2805736 ]
#####################################################


#Implementation of linear classifier

The implementation of the linear classifier should be constructed as a class that initializes a weight matrix based on the shape of the input and number of classes and type of loss computed (SVM/Softmax). It should have methods for training (updating the weights) and inference (predicting classes from scores).

Note!

The training function should propably have a "batch_size" parameter such that training data is iterated in batches so as to not run out of memory. Use np.array_split to divide data into n batches of "batch_size".

psuedo code example:


    class LinearClassifier:
      def __init__(self, input_dim, num_classes, loss_type='softmax'):

          self.W = np.random.randn(input_dim, num_classes) * 0.01
          self.loss_type = loss_type

          method train()

          method softmax_loss()

          method svm_loss()

          method predict()

In [ ]:

##################################
#Implement linear classifier class
##################################


#Train the classifier once on the training set
It might be a good idea to have the training function of you model return a "history" or similar object holding the values of the loss at each m steps of the training. Afterwards the history object (for instance a list) can be plottet to visualize how training progressed.

Calling the training function would thus look something like this:

    classifier = LinearClassifier(input_dim=28*28*3+1,num_classes=8, loss_type='svm')

    history = classifier.train(params)

    plot(history)

    #make predictions on the validation data
    pred = classifier.predict(X)

    #calculate the accuracy

In [ ]:
##########################################
#Train the classifier and plot the results
##########################################

#Optimize SVM classifier
We start by optimizing the the classifier with the svm loss function.

Using the training and validation sets run training loops with different setting for learning rate and regularization strength. For each setting, iterate over the training set n times updating the weight matrix based on the gradients and evaluate performance on the validation set every m iterations. Save the best classifier object in a variable "best_svm".


E.g.

    best_svm = None
    best_accuracy = 0

    # These values are only given as examples. In your own loop
    # you should test more in order to optimize the classifier

    learning_rates = [1e-2,1e-4,1e-6]
    regularization_strenghts = [1,1e-2,1e-1,1e2]

    num_iters = 1000
    batch_size = 200

    for lr in learning_rates:
      for reg in regularizations_strenghts:

        svm = LinearClassifier(input_dim=28*28*3+1,num_classes=8,loss_type='svm')

        svm.train(params)

        pred = svm.predict(X)

        #calculate accuracy

        
        if accuracy > best_accuracy:
          best_svm = svm
          best_accuracy = accuracy


! It may take a long time for the model to converge. Therefor, the hyperparameter search in this notebook could be used to identify a reasonable set of hyperparameters and then the model should be trained for longer using these. Using the full training dataset to maximize performance
        


In [ ]:
###########################
#Put optimization code here
###########################



I could be a good idea to record as much informartion as possible from the optimization. E.g. loss as function of hyperparameters. This can be used for visualizing results of the tuning and included in the assignment 1 report.

#visualize the weights
Depending on how you implemented the classifier it is possible to visualize the learned weights. I the classifier is well optimized you may notice the sort of class templates encoded in the weight matrix.



In [ ]:
 #Visualization of learned weights. This may or may not work depending on how your classifier is implemented.
w = best_svm.W[:-1,:] # strip out the bias
w = w.reshape(28, 28, 3, 8)
w_min, w_max = np.min(w), np.max(w)
classes = ['0', '1', '2', '3', '4', '5', '6', '7']
for i in range(8):
    plt.subplot(1, 8, i + 1)

    # Rescale the weights to be between 0 and 255
    wimg = 255.0 * (w[:, :, :, i].squeeze() - w_min) / (w_max - w_min)
    plt.imshow(wimg.astype('uint8'))
    plt.axis('off')
    plt.title(classes[i])



You can compare the learned weights to a set of randomly initialized weights

In [ ]:
#Compare the trained weights to a set of random initialized weights
w = np.random.randn(28*28*3+1, num_classes) * 0.001
w = w[:-1,:] # strip out the bias
w = w.reshape(28, 28, 3, 8)
w_min, w_max = np.min(w), np.max(w)
classes = ['0', '1', '2', '3', '4', '5', '6', '7']
for i in range(8):
    plt.subplot(1, 8, i + 1)

    # Rescale the weights to be between 0 and 255
    wimg = 255.0 * (w[:, :, :, i].squeeze() - w_min) / (w_max - w_min)
    plt.imshow(wimg.astype('uint8'))
    plt.axis('off')
    plt.title(classes[i])

#Optimize softmax classifier
Repeat the steps above but using the linear classifier with softmax loss. Report results for both in assignment 1

In the hand-in for assignment 1 you should be able to explain the implementation of the loss and gradients for the two types of linear classifiers. Therefore it might be good to include relevant code snippets.

The Hand in should include the tuning results/graphs (train and validation to discuss overfitting potential) and test set results think about metrics that make sense to calculate and include. E.g confusion matrix.